### roadmap

1. **Estimate average transactions per year, $f$**

   * **Logic**: For each customer $c$, count their orders ($O_c$) and compute their tenure in years ($Y_c = (\text{last order date} - \text{first order date})/365$).
   * **Formula**:

     $$
       f \;=\;\frac{\sum_c (O_c - 1)}{\sum_c Y_c}
     $$

     This is the total number of *repeat intervals* divided by total customer‐years.

2. **Convert annual discount into a per-purchase rate, $r$**

   * **Logic**: We want the equivalent discount each time a purchase occurs, given an annual rate $r_{\rm annual}$.
   * **Formula**:

     $$
       1 + r \;=\;(1 + r_{\rm annual})^{1/f}
       \quad\Longrightarrow\quad
       r \;=\;(1 + r_{\rm annual})^{1/f}\;-\;1.
     $$

3. **Sequence each customer’s purchases and choose horizon $N$**

   * **Logic**: Order each customer’s transactions chronologically and label them $t=1,2,\dots$. The furthest any customer goes defines the maximum depth $N$.
   * **Formula**:

     $$
       t_{c,i} = i,\quad i\;\text{th purchase of customer }c;\qquad
       N = \max_{c,i}(t_{c,i}).
     $$

4. **Compute mean order value at transaction $t$, $M_t$**

   * **Logic**: For the $t$ th purchase across all customers who have made at least $t$ purchases, average their order revenues.
   * **Formula**:

     $$
       M_t \;=\;\frac{1}{\#\{c: t_{c,\bullet}=t\}}
                   \sum_{c:\,t_{c,\bullet}=t} 
                   \text{LineDollars}_{c,t}.
     $$

5. **Compute conditional purchase probability at $t$, $r_t$**

   * **Logic**: Let $\mathrm{reach}_t$ = number of customers who made at least $t$ purchases. Then
   * **Formulas**:

     $$
       r_1 = 1,
       \quad
       r_t = \frac{\mathrm{reach}_t}{\mathrm{reach}_{t-1}}
       \quad(t>1),
     $$

     where $\mathrm{reach}_t = \#\{c:\max_i t_{c,i}\ge t\}$.

6. **Estimate average acquisition cost per customer, $\overline{AC}$**

   * **Logic**: For each customer, count their catalog and e-mail contacts *before* first purchase, multiply by unit costs $u_C$, $u_E$, then average across all customers.
   * **Formulas**:

     $$
       AC_c = u_C \,n_c^{\rm catalog} \;+\; u_E \,n_c^{\rm email},
       \quad
       \overline{AC} = \frac1C \sum_{c=1}^C AC_c.
     $$

7. **Compute overall CLV**

   * **Logic**: The expected discounted revenue per acquired customer (common to all) minus the average acquisition cost.
   * **Formula**:

     $$
       \boxed{
         \mathrm{CLV}
         \;=\;
         -\,\overline{AC}
         \;+\;\sum_{t=1}^{N}
           \frac{r_t\,M_t}{(1 + r)^{\,t-1}}
       }
     $$
   * Here
   * $r$ = per-purchase discount (Step 2),
   * $r_t$ = conditional repurchase prob. (Step 5),
   * $M_t$ = mean order value at depth $t$ (Step 4),
   * $N$ = maximum transaction depth (Step 3).

---

With these seven steps—each driven entirely by your orders and contacts data—you obtain a single, data-grounded CLV estimate without any hand-tuned inputs.


### implementation

#### step 0, Input required parameters

In [4]:
r_annual = 0.10        # e.g. 10% annual discount
u_C      = 2.00        # per-catalog cost
u_E      = 0.10        # per-email cost

final notes:
No returns data are provided, so revenue is gross
The documentation warns that customers were sampled at different rates for anonymity. Average CLV for this teaching sample may not equal the true population average.

#### Step 1. Estimate average transactions per year, f

In [6]:
sql = """CREATE VIEW cust_stats AS
SELECT
    l."Cust_ID",
    l."OrderNum",
    l."LineDollars",
    o."OrderDate",
    o."OrderMethod",
    c."AcqDateFormatted"
FROM "lines" l
JOIN "orders" o
    ON l."Cust_ID" = o."Cust_ID" AND l."OrderNum" = o."OrderNum"
JOIN "summary" c
    ON l."Cust_ID" = c."Cust_ID";"""


with engine.begin() as conn:
    conn.execute(text(sql))


In [5]:
sql = """
WITH cust_stats AS (
  SELECT
    "Cust_ID",
    COUNT(*)            AS num_orders,
    MIN("OrderDate")      AS first_date,
    MAX("OrderDate")      AS last_date
  FROM orders
  GROUP BY "Cust_ID"
),
agg AS (
  SELECT
    SUM(num_orders - 1)                                       AS total_intervals,
    SUM(EXTRACT(EPOCH 
        FROM (last_date - first_date)) / 86400.0) / 365.0     AS total_years
  FROM cust_stats
)
SELECT
  total_intervals / NULLIF(total_years,0)  AS f_est
FROM agg;
"""
df_f  = pd.read_sql_query(sql, engine)
f_est = df_f.at[0, "f_est"]
print(f"Estimated f (txns/year) = {f_est:.3f}")


Estimated f (txns/year) = 1.450


#### Step 2. Convert annual discount to per-purchase rate,

In [6]:

# compute per‐purchase discount rate
r = (1 + r_annual)**(1.0 / f_est) - 1

print(f"Per-purchase discount rate r = {r:.4f}")


Per-purchase discount rate r = 0.0679


#### Step 3. Sequence each customer’s orders (𝑡) and find horizon 𝑁

In [ ]:
sql = """CREATE OR REPLACE VIEW sequenced_orders AS
SELECT
  "Cust_ID",
  "OrderNum",
  "LineDollars",
  "OrderDate",
  ROW_NUMBER() OVER (
    PARTITION BY "Cust_ID"
    ORDER BY "OrderDate"
  ) AS t
FROM cust_stats;

SELECT MAX(t) AS N FROM sequenced_orders;"""


with engine.begin() as conn:
    conn.execute(text(sql))



In [9]:
df = pd.read_sql_query("SELECT MAX(t) AS N FROM sequenced_orders;", engine)
N = df.at[0, 'n']
print(f"Horizon N = {N}")


Horizon N = 973


#### Step 4. Compute mean order value at transaction $t$, $M_t$

​


In [28]:
sql = """CREATE OR REPLACE VIEW Mt AS
SELECT
  t,
  AVG("LineDollars") AS M_t,
  COUNT(*)         AS n_t
FROM sequenced_orders
GROUP BY t
ORDER BY t;"""

with engine.begin() as conn:
    conn.execute(text(sql))

In [29]:
df_M = pd.read_sql_query("SELECT * FROM Mt;", engine)
print(df_M.head())


   t        m_t     n_t
0  1  36.263360  100051
1  2  30.860016   70046
2  3  28.985962   53446
3  4  27.854060   42443
4  5  27.287394   34393


#### Step 5. Compute conditional repurchase probabilities $ r_t $

​


In [12]:
sql = """CREATE OR REPLACE VIEW rt AS
WITH cust_max AS (
  SELECT
    "Cust_ID",
    MAX(t) AS max_t
  FROM sequenced_orders
  GROUP BY "Cust_ID"
),
overall AS (
  SELECT MAX(max_t) AS overall_max
  FROM cust_max
),
reach AS (
  -- note that generate_series is in the FROM clause
  SELECT
    gs.t,
    (
      SELECT COUNT(*) 
      FROM cust_max 
      WHERE max_t >= gs.t
    ) AS reach_t
  FROM overall
  CROSS JOIN LATERAL
    generate_series(1, overall.overall_max) AS gs(t)
)
SELECT
  t,
  CASE
    WHEN t = 1 THEN 1.0
    ELSE reach_t::float / LAG(reach_t) OVER (ORDER BY t)
  END AS r_t
FROM reach
ORDER BY t;
"""


with engine.begin() as conn:
    conn.execute(text(sql))

In [13]:
df_r_t = pd.read_sql_query("SELECT * FROM rt;", engine)
print(df_r_t.head())


   t       r_t
0  1  1.000000
1  2  0.700103
2  3  0.763013
3  4  0.794129
4  5  0.810334


Step 6. Estimate each customer’s acquisition cost $ AC _ t $
​


In [20]:
from sqlalchemy import text



sql = f"""
CREATE OR REPLACE VIEW AC_tbl AS
WITH first_order AS (
  SELECT "Cust_ID",
         MIN("OrderDate") AS first_date
  FROM cust_stats
  GROUP BY "Cust_ID"
),
pre AS (
  SELECT
    c."Cust_ID",
    SUM(CASE WHEN "ContactType" = 'C' THEN 1 ELSE 0 END) AS cat_cnt,
    SUM(CASE WHEN "ContactType" = 'E' THEN 1 ELSE 0 END) AS email_cnt
  FROM contacts c
  JOIN first_order f
    USING ("Cust_ID")
  WHERE c."ContactDate" < f.first_date
  GROUP BY c."Cust_ID"
)
SELECT
  "Cust_ID",
  {u_C} * cat_cnt + {u_E} * email_cnt AS AC_c
FROM pre;
"""

with engine.begin() as conn:
    conn.execute(text(sql))


In [21]:
df_AC = pd.read_sql_query("SELECT * FROM AC_tbl", engine)
print(df_AC.head())


   Cust_ID  ac_c
0    24436   8.1
1   339962   4.0
2   353683   6.0
3   363725   2.0
4   866490  50.0


#### Step 7. Compute overall CLV

In [23]:
AC_avg = pd.read_sql_query(
    "SELECT AVG(AC_c) FROM AC_tbl;",
    engine
).squeeze()


In [30]:
sql = f"""
SELECT
  SUM(r_t * M_t / POWER(1 + {r}, t - 1)) AS S_val
FROM Mt
JOIN rt USING (t);
"""

df_S = pd.read_sql_query(sql, engine)
S_val = df_S.iloc[0, 0]
print("S =", S_val)


S = 361.21965726708294


In [31]:
# 3) overall CLV
CLV_overall = -AC_avg + S_val
print(f"Overall CLV = ${CLV_overall:,.2f}")

Overall CLV = $354.96
